## Parameters

In [33]:
input_data='./input_data/'
output_data='./output_data/'

## Libraries

In [34]:
import numpy as np
import pandas as pd
import datetime

In [35]:
cc_case='WC1'

Load the worst case impact table. 

WC1 and WC2 are the two worst case impact predictions giving reduction in flow by month for 2070.
- *WC1*: Hamududu & Killingtviet 2016
- *WC2*: Harrison & Whittington, 2002

In [36]:
cc_cases=pd.read_csv(input_data + 'climate_change_worst_cases.csv').set_index('Month')
cc_cases

,WC1,WC2
Month,,
10,-0.16,-0.243920
11,-0.20,-0.296011
12,-0.17,-0.396367
1,-0.18,-0.326546
2,-0.16,-0.324661
3,-0.41,-0.395765
4,-0.29,-0.386358
5,-0.24,-0.362866
6,-0.24,-0.350233


In [37]:
daily_flow=pd.read_csv(output_data + 'ngonye_flow_daily.csv')

daily_flow=daily_flow.set_index(pd.DatetimeIndex(daily_flow['Date']))
daily_flow=daily_flow.drop(['LaggedDate','VicFalls','Conversion','Flow_difference','Volume','MonthId','Date','WaterMonth','WaterDay','WaterWeek','Exceedance'],axis=1)
daily_flow.head(4)

,Flow,Year,Month,Day,WaterYear
Date,,,,,
1924-10-01,111.133124,1924,10,1,1924
1924-10-02,111.133124,1924,10,2,1924
1924-10-03,111.133124,1924,10,3,1924
1924-10-04,111.133124,1924,10,4,1924


In [59]:
sims=pd.DataFrame(np.arange(1,61,1),columns=['SimNo'])
sims['StartDate']=pd.date_range(start=datetime.date(1933, 10, 1), periods=60, freq=pd.tseries.offsets.DateOffset(years=1))
sims=sims.set_index('SimNo')
sims

,StartDate
SimNo,
1,1933-10-01
2,1934-10-01
3,1935-10-01
4,1936-10-01
5,1937-10-01
6,1938-10-01
7,1939-10-01
8,1940-10-01
9,1941-10-01


Calculate the Climate Change Rate which is 0% in 1990 and rises to 100% in 2070. This sets the proportion of the overall climate change impact (by 2070) which is assumed to apply to any given date.

In [39]:
daily_flow['CC_Factor']=daily_flow.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
daily_flow['Impact_2070']=daily_flow.join(cc_cases,on='Month')[cc_case]
daily_flow['Flow_Adjustment']=(1+daily_flow['CC_Factor']*(-daily_flow['Impact_2070']))
daily_flow['Flow_Adjusted']=daily_flow['Flow']*daily_flow['Flow_Adjustment']
daily_flow

,Flow,Year,Month,Day,WaterYear,CC_Factor,Impact_2070,Flow_Adjustment,Flow_Adjusted
Date,,,,,,,,,
1924-10-01,111.133124,1924,10,1,1924,0.0000,-0.16,1.00000,111.133124
1924-10-02,111.133124,1924,10,2,1924,0.0000,-0.16,1.00000,111.133124
1924-10-03,111.133124,1924,10,3,1924,0.0000,-0.16,1.00000,111.133124
1924-10-04,111.133124,1924,10,4,1924,0.0000,-0.16,1.00000,111.133124
1924-10-05,111.133124,1924,10,5,1924,0.0000,-0.16,1.00000,111.133124
...,...,...,...,...,...,...,...,...,...
2017-09-26,203.017496,2017,9,26,2016,0.3375,-0.14,1.04725,212.610072
2017-09-27,203.017496,2017,9,27,2016,0.3375,-0.14,1.04725,212.610072
2017-09-28,203.017496,2017,9,28,2016,0.3375,-0.14,1.04725,212.610072


In [40]:
daily_flow.loc['2015-10-10']

Flow                203.017496
Year               2015.000000
Month                10.000000
Day                  10.000000
WaterYear          2015.000000
CC_Factor             0.312500
Impact_2070          -0.160000
Flow_Adjustment       1.050000
Flow_Adjusted       213.168370
Name: 2015-10-10 00:00:00, dtype: float64

In [41]:
tmp=daily_flow.reset_index().head(4018)[['Date','Flow','Flow_Adjusted']].copy()
tmp['Date']=pd.date_range(start=datetime.date(2017, 10, 1), periods=4018, freq='D')
tmp=tmp.set_index('Date')
tmp['Day']=tmp.index.day
tmp['Month']=tmp.index.month
tmp['Year']=tmp.index.year
tmp['WaterYear']=tmp.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
tmp

,Flow,Flow_Adjusted,Day,Month,Year,WaterYear
Date,,,,,,
2017-10-01,111.133124,111.133124,1,10,2017,2017.0
2017-10-02,111.133124,111.133124,2,10,2017,2017.0
2017-10-03,111.133124,111.133124,3,10,2017,2017.0
2017-10-04,111.133124,111.133124,4,10,2017,2017.0
2017-10-05,111.133124,111.133124,5,10,2017,2017.0
...,...,...,...,...,...,...
2028-09-26,259.369380,259.369380,26,9,2028,2027.0
2028-09-27,259.369380,259.369380,27,9,2028,2027.0
2028-09-28,254.145972,254.145972,28,9,2028,2027.0


In [42]:
daily=daily_flow.append(tmp)
daily=daily.drop(['CC_Factor','Impact_2070','Flow_Adjustment'],axis=1)
daily

,Flow,Year,Month,Day,WaterYear,Flow_Adjusted
Date,,,,,,
1924-10-01,111.133124,1924,10,1,1924.0,111.133124
1924-10-02,111.133124,1924,10,2,1924.0,111.133124
1924-10-03,111.133124,1924,10,3,1924.0,111.133124
1924-10-04,111.133124,1924,10,4,1924.0,111.133124
1924-10-05,111.133124,1924,10,5,1924.0,111.133124
...,...,...,...,...,...,...
2028-09-26,259.369380,2028,9,26,2027.0,259.369380
2028-09-27,259.369380,2028,9,27,2027.0,259.369380
2028-09-28,254.145972,2028,9,28,2027.0,254.145972


In [43]:
future=pd.DataFrame(index=pd.date_range(start='10/1/2017', end='09/30/2053', freq='D'))
future['Year']=future.index.year
future['Month']=future.index.month
future['CC_Factor']=future.apply(lambda x: (0 if x['Year']<1991 else (x['Year']-1990)/80),axis=1)
future

,Year,Month,CC_Factor
2017-10-01,2017,10,0.3375
2017-10-02,2017,10,0.3375
2017-10-03,2017,10,0.3375
2017-10-04,2017,10,0.3375
2017-10-05,2017,10,0.3375
...,...,...,...
2053-09-26,2053,9,0.7875
2053-09-27,2053,9,0.7875
2053-09-28,2053,9,0.7875
2053-09-29,2053,9,0.7875


In [61]:

def calc_flows(sim_no):
    start_date=sims.loc[sim_no]['StartDate']

    future['SampleDate']=pd.date_range(start=start_date, periods=13149, freq='D')
    future['SampleFlow']=future.join(daily,on='SampleDate',rsuffix='_daily')['Flow_Adjusted']
    future['Impact_2070']=future.join(cc_cases,on='Month')[cc_case]
    future['Flow' + str(sim_no)]=future['SampleFlow']*(1+future['Impact_2070']*future['CC_Factor'])
 

In [68]:
for sim in sims.itertuples():
    calc_flows(sim.Index)
    
future

,Year,Month,CC_Factor,SampleDate,SampleFlow,Impact_2070,Flow,Flow1,Flow3,Flow2,...,Flow51,Flow52,Flow53,Flow54,Flow55,Flow56,Flow57,Flow58,Flow59,Flow60
2017-10-01,2017,10,0.3375,1992-10-01,172.985373,-0.16,175.961255,175.961255,240.422090,223.189910,...,207.106491,199.825912,214.984774,245.363434,226.678548,233.814558,259.747311,211.959765,187.303752,163.644163
2017-10-02,2017,10,0.3375,1992-10-02,171.191378,-0.16,175.961255,175.961255,240.422090,219.900733,...,207.106491,199.825912,214.984774,240.422090,226.678548,233.814558,259.747311,212.346318,187.303752,161.947044
2017-10-03,2017,10,0.3375,1992-10-03,171.191378,-0.16,175.961255,175.961255,235.966661,214.984774,...,207.106491,196.684796,211.959765,233.814558,226.678548,233.814558,259.045538,208.160306,187.303752,161.947044
2017-10-04,2017,10,0.3375,1992-10-04,171.191378,-0.16,173.087025,173.087025,233.814558,211.959765,...,203.447769,196.684796,211.959765,233.814558,226.678548,230.397229,259.045538,207.106491,187.303752,161.947044
2017-10-05,2017,10,0.3375,1992-10-05,171.191378,-0.16,173.087025,173.087025,226.678548,211.419041,...,203.447769,196.684796,211.419041,230.397229,226.678548,230.397229,259.045538,207.106491,187.303752,161.947044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053-09-26,2053,9,0.7875,2028-09-26,259.369380,-0.14,387.937245,387.937245,297.335011,304.745173,...,191.350584,173.918166,234.117333,178.994480,236.666682,236.666682,236.666682,173.456412,219.911737,230.773906
2053-09-27,2053,9,0.7875,2028-09-27,259.369380,-0.14,383.826433,383.826433,294.660513,304.745173,...,184.989743,173.456412,230.773906,174.272966,236.666682,226.126379,230.773906,171.307455,213.200040,230.773906
2053-09-28,2053,9,0.7875,2028-09-28,254.145972,-0.14,383.826433,383.826433,290.149898,301.941999,...,183.037465,171.307455,230.773906,174.272966,236.666682,221.935874,230.773906,166.273311,211.315967,226.126379
2053-09-29,2053,9,0.7875,2028-09-29,254.145972,-0.14,380.564113,380.564113,287.046605,301.941999,...,180.622439,171.307455,226.126379,174.272966,234.117333,221.935874,226.126379,166.273311,211.315967,226.126379
